# Stage three hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [2]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 10
n_trials = 150
importance_type = "split"
n_ehvi_candidates = 12
trait = "lifespan"#"lifespan"
opt_metrics = "r2"
genes_num=10
stage_num=3
debug_local = True #to use local version

In [3]:
suffix=f"_{stage_num}"

In [4]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/data/miniconda3/envs/yspecies/lib/python38.zip', '/data/miniconda3/envs/yspecies/lib/python3.8', '/data/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/anton/.ipython']


In [5]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.tuning import Tune, MultiObjectiveResults
from yspecies.explanations import FeatureSummary, ShapSelector
from yspecies.selection import CrossValidator
import optuna
from optuna import Study, Trial

In [7]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: agg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [8]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [9]:
from loguru import logger

logger.add(locations.logs / f"optimization_stage{suffix}_errors.log", backtrace=True, diagnose=True)
logger.add(locations.logs / f"optimization_stage{suffix}.log", rotation="12:00")     # New file is created each day at noon

2

In [10]:
data = ExpressionDataset.from_folder(locations.interim.stage(stage_num-1) / "input")
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 154)","(154, 37)",38,408,"(154, 2)","(38, 18)"


### Loading genes from the ranked selection ###

In [11]:
intersections = locations.output.intersections
ranked = pd.read_csv(intersections / "intersections_ranked.tsv", sep="\t", index_col="reference_gene")
print(ranked.shape)
ranked.head(10)

(152, 23)


,symbol,rank,ranks_sum,repeats_rank,shap_rank,kendall_tau_rank,frequency_rank,tissue_linear_rank,genage_rank,repeats_all,repeats,mean_abs_shap,mean_kendall_tau,frequency,max_linear_r2,genage_count,direction,repeats_lifespan_stage_1,mean_abs_shap_lifespan_stage_1,mean_kendall_tau_lifespan_stage_1,direction_lifespan_stage_1,other_life_history_traits,organ
reference_gene,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000188747,NOXA1,1.000,46.000,2.000,12.000,16.000,1.000,10.000,6.000,19.000,10.000,0.507,0.697,1.000,0.627,0.000,strong pro,9.000,0.413,0.689,strong pro,"mtGC (pro | 8 | 0.05 | 0.649), gestation_days ...","Brain (pro r^2=0.6271), Kidney (pro r^2=0.6017..."
ENSG00000170835,CEL,2.000,54.000,1.000,7.000,7.000,6.000,33.000,6.000,20.000,10.000,0.849,0.746,0.600,0.481,0.000,strong pro,10.000,1.068,0.693,strong pro,NaN,Brain (pro r^2=0.4815)
ENSG00000136436,CALCOCO2,3.000,63.000,7.000,27.000,5.000,7.000,18.000,6.000,14.000,10.000,0.307,0.763,0.420,0.559,0.000,strong pro,4.000,0.074,0.722,weak pro,NaN,"Lung (pro r^2=0.5586), Brain (pro r^2=0.5369),..."
ENSG00000198663,C6orf89,4.000,64.000,10.000,35.000,1.000,2.000,12.000,6.000,10.000,10.000,0.226,-0.788,0.940,0.616,0.000,strong anti,0.000,nan,nan,NaN,metabolic_rate (anti | 6 | 0.0 | -0.5674),"Heart (anti r^2=0.6159), Liver (anti r^2=0.4011)"
ENSG00000172531,PPP1CA,5.000,69.000,5.000,42.000,3.000,20.000,13.000,6.000,16.000,10.000,0.165,-0.782,0.000,0.610,0.000,strong anti,6.000,0.093,-0.761,anti,gestation_days (strong anti | 9 | 1.28 | -0.7678),"Brain (anti r^2=0.6099), Heart (anti r^2=0.4668)"
ENSG00000006282,SPATA20,6.000,75.000,8.000,25.000,31.000,16.000,5.000,6.000,13.000,10.000,0.311,0.649,0.080,0.675,0.000,strong pro,3.000,0.148,0.608,weak pro,"mtGC (strong pro | 9 | 0.24 | 0.6937), gestati...","Brain (pro r^2=0.675), Heart (pro r^2=0.6381),..."
ENSG00000142002,DPP9,7.000,76.000,4.000,26.000,11.000,11.000,29.000,6.000,17.000,10.000,0.309,-0.717,0.300,0.507,0.000,strong anti,7.000,0.101,-0.716,anti,NaN,"Lung (anti r^2=0.5074), Heart (anti r^2=0.3048)"
ENSG00000129187,DCTD,8.000,78.000,8.000,32.000,26.000,8.000,6.000,6.000,13.000,10.000,0.253,0.674,0.400,0.666,0.000,strong pro,3.000,0.149,0.632,weak pro,mtGC (weak pro | 4 | 0.01 | 0.6756),"Liver (pro r^2=0.6659), Heart (pro r^2=0.4353)..."
ENSG00000165501,LRR1,9.000,82.000,2.000,19.000,14.000,9.000,41.000,6.000,19.000,10.000,0.366,0.715,0.340,0.459,0.000,strong pro,9.000,0.177,0.688,strong pro,NaN,"Heart (pro r^2=0.4585), Lung (pro r^2=0.4286),..."


In [12]:
def data_genes(genes: List[str]):
    return data.by_genes.collect(lambda gs: gs.loc[genes])

def data_head(num: int):
    return data_genes(ranked.head(num).index.to_list())

In [13]:
not_validated_species = data.min_max_trait(trait)
not_validated_species

['Mus_caroli', 'Homo_sapiens']

In [14]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = trait, #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent",
    not_validated_species = not_validated_species
)
selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,"['Mus_caroli', 'Homo_sapiens']"


In [15]:
top_genes_data = data_head(genes_num)
top_genes_data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 10)","(10, 37)",38,408,"(10, 2)","(38, 18)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [16]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

In [17]:
sampler_tpe = optuna.multi_objective.samplers.MOTPEMultiObjectiveSampler(
        n_ehvi_candidates=n_ehvi_candidates,
        seed=0
    )

MOTPEMultiObjectiveSampler is experimental (supported from v2.3.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.


In [18]:
def load_study(trait: str, suffix: str = suffix, sampler = sampler_tpe):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+suffix+".sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], 
                                                     storage = storage, 
                                                     study_name = f"{trait}_r2_huber_kendall", 
                                                     load_if_exists = True
                                                    )

study = load_study(trait, suffix)
study
#metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
#metrics, params

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan_3.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-11-14 18:22:11,602] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [19]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [20]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("cross_validation", CrossValidator()),
        ("shap_computation", ShapSelector())
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [21]:
p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
results = p.fit_transform((top_genes_data, selection))
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
2020-11-14 18:22:13.210 | INFO     | yspecies.selection:fit:157 - =

0
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1


Early stopping is not available in dart mode
2020-11-14 18:29:06.908 | INFO     | yspecies.selection:fit:161 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']


[250]	valid_0's l2: 202.625	valid_0's huber: 6.65293	valid_0's l1: 7.76657


2020-11-14 18:37:23.332 | INFO     | yspecies.selection:fit:161 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']


...

[250]	valid_0's l2: 199.458	valid_0's huber: 5.13112	valid_0's l1: 6.03824
[250]	valid_0's l2: 125.075	valid_0's huber: 5.20737	valid_0's l1: 6.1414


2020-11-14 18:55:29.966 | INFO     | yspecies.explanations:transform:741 - Metrics: 
Metrics(R2=0.680407820306089, MAE=6.982194207552144, MSE=226.2451225890496, huber=5.960562619935221)
2020-11-14 18:55:29.989 | INFO     | yspecies.selection:fit:157 - ===== fitting models with seed 1 =====
2020-11-14 18:55:29.994 | INFO     | yspecies.selection:fit:158 - PARAMETERS:
{'objective': 'regression', 'metric': {'mse', 'huber', 'mae'}, 'verbosity': -1, 'boosting_type': 'dart', 'lambda_l1': 2.9902884884096714, 'lambda_l2': 0.6136139209038782, 'max_leaves': 16, 'max_depth': 5, 'feature_fraction': 0.506094745993802, 'bagging_fraction': 0.6549913402207208, 'learning_rate': 0.08554380802136563, 'min_data_in_leaf': 5, 'drop_rate': 0.2592906278504832, 'verbose': -1, 'seed': 0}
2020-11-14 18:55:30.003 | INFO     | yspecies.selection:fit:161 - SEED: 1 | FOLD: 0 | VALIDATION_SPECIES: ['Felis_catus', 'Ursus_americanus']


1
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1


Early stopping is not available in dart mode
2020-11-14 19:09:55.154 | INFO     | yspecies.selection:fit:161 - SEED: 1 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Vombatus_ursinus']


[250]	valid_0's l2: 154.758	valid_0's huber: 4.63431	valid_0's l1: 5.51668


In [ ]:
results.best_trial_r2().values

In [ ]:
print("first optimization was finished")

In [ ]:
results.best_trial_r2().values

In [19]:
#optuna.multi_objective.visualization.plot_pareto_front(study, include_dominated_trials =True)